In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import itertools
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
from sklearn import metrics

from sqlalchemy import create_engine
import datetime 
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib
pd.options.display.max_columns=300
pd.options.display.max_rows=100
# from TurbineTimeSeries.storage import MachineDataStore
plt.rcParams["figure.figsize"] = (14,8)
#from TurbineTimeSeries.transformations import PCA, StandardScaler, DropCols, DropSparseCols, LeftJoin
%matplotlib inline

In [2]:
# model_number = 2

# store = MachineDataStore('.config')

# # model_data_hr = (store.query(model_number,'1hr')
# #                  .not_null(['timestamp','psn'])
# # #                  .exclude_psn([44,52,54,70])
# #                  .execute())

# model_data_min = (store.query(model_number,'10min')
#                   .not_null(['timestamp','psn'])
#                   .exclude_psn([44,52,54,70])
#                   .execute())

# model_data_min.head()
# model1_1hr = pd.read_csv('../../../data/raw_data_model1.csv',index_col=0)
# model2_1hr = pd.read_csv('../../../data/raw_data_model2.csv',index_col=0)

# model1_10min = pd.read_csv('../../../data/raw_data_model1_10min.csv',index_col=0)
model2_10min = pd.read_csv('../../../data/raw_data_model2_10min.csv')#,index_col=0)

## convert all to timestamps
# model1_1hr['timestamp'] = model1_1hr['timestamp'].apply(lambda x: pd.Timestamp(x))
# model2_1hr['timestamp'] = model2_1hr['timestamp'].apply(lambda x: pd.Timestamp(x))
# model1_10min['timestamp'] = model1_10min['timestamp'].apply(lambda x: pd.Timestamp(x))
model2_10min['timestamp'] = model2_10min['timestamp'].apply(lambda x: pd.Timestamp(x))


model2_10min = model2_10min.sort_values(by=['psn','timestamp'],ascending=[True,True])

# print('Shape of model1 1hr data: ', model1_1hr.shape)
# print('Shape of model2 1hr data: ', model2_1hr.shape)
# print('Shape of model1 10min data: ', model1_10min.shape)
print('Shape of model2 10min data: ', model2_10min.shape)

Shape of model2 10min data:  (1602326, 76)


In [9]:
def shutdown_labeler(df,eng_st_col= 'sum_eng_st'):
    shutdown_df = pd.DataFrame(columns = df.columns.values)
    shutdown_df['shutdown_flag']=0
    for psn in df['psn'].unique():
        tempdf = df[df['psn']==psn]
        shutdown_flag = np.diff(tempdf[eng_st_col])
        tempdf['shutdown_flag'] = np.array(np.append(shutdown_flag,[0]),dtype=bool) ## cast everything to boolean
        shutdown_df = shutdown_df.append(tempdf)
    return(shutdown_df)

In [10]:
model2_10min_with_shutdowns = shutdown_labeler(model2_10min)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
model2_10min_with_shutdowns.head()

,timestamp,psn,id,lo_c_dp1,f_c_dp1,f_c_dp2,f_c_dp5,pe_c_dt1,g_c_dt1,g_c_dt2,lo_c_dt5,c_dt5_1,c_dt5_2,c_dt5_3,c_dt5_4,c_dt5_5,c_dt5_6,sc_c_pct_e1,pe_c_pos_e1,f_c_pos_e2,f_c_pos_e1,c_c_t5_1,c_c_t5_2,c_c_t5_3,t5_s1,pe_p1,f_cmd1,pe_t1,perf_pow,sum_eng_h,lo_p1,b_p1,g_t1,t5_6,g_t5,g_t4,v_d_1b,g_cur1,g_cur4,g_cur3,lo_t9,f_cmd2,lo_t6,t5_5,t5_2,t5_3,t5_1,ngp,nt5,pcd,v_acc1,sc_pct2,g_pow1,pe_for1,sc_pct1,pe_cmd1,pe_pos1,g_cur2,t1_1,sum_esn,t5_4,f_p2,t5_a,sum_eng_st,lo_dp1,v_d_3b,f_p7,f_p1,v_d_2b,lo_t5,g_t2,f_t1,sum_enr,g_t3,g_pct1,f_pos1,shutdown_flag
515426,2015-12-05 00:00:00.000000,34,1211745,0.230272,0.155038,0.368851,0.185603,0.541111,0.030,0.002,0.438823,-0.068389,0.069944,-0.133055,-0.011389,0.057278,0.085611,0.127131,-0.042950,-0.02556,-0.01372,0.833632,0.374857,0.225570,0.2624,0.602404,0.133358,0.489697,0.615316,0.088200,0.820926,0.09640,0.854857,0.850754,0.4340,0.4776,0.262457,0.319687,0.326496,0.320871,0.898421,0.607376,0.987333,0.845088,0.847621,0.807021,0.819954,0.999599,1.0,0.286693,0.199887,0.959319,-0.9751,0.223349,0.0,0.868627,0.886859,0.322344,0.1605,72.0,0.831354,0.623482,0.833632,111.0,0.223076,0.23472,14.759244,0.617559,0.209029,0.806087,0.846857,0.5450,13.472658,0.4536,0.36426,0.134215,0
515427,2015-12-05 00:10:00.000004,34,1211746,0.279799,0.236221,0.221298,0.186310,0.551111,0.068,0.046,0.472941,-0.062445,0.066222,-0.128444,-0.017778,0.054556,0.087889,0.082469,-0.042950,-0.02698,-0.01170,0.833441,0.370857,0.228597,0.2596,0.600000,0.133484,0.495758,0.614241,0.088200,0.844706,0.11170,0.853714,0.851019,0.4636,0.4664,0.265029,0.320982,0.326763,0.321004,0.897895,0.607689,0.973333,0.844353,0.846686,0.807753,0.820952,0.999699,1.0,0.285623,0.204233,0.960060,-0.9750,0.218476,0.0,0.868627,0.886859,0.322902,0.1610,72.0,0.829886,0.624072,0.833441,111.0,0.276924,0.24372,14.747893,0.618634,0.206800,0.809565,0.847429,0.5450,13.575190,0.4840,0.36368,0.134215,0
515428,2015-12-05 00:19:59.999998,34,1211747,0.276643,0.168140,0.351799,0.186543,0.536667,0.084,0.062,0.465882,-0.064889,0.072778,-0.128222,-0.014222,0.052444,0.082111,0.102534,-0.052448,-0.03324,-0.01392,0.832753,0.360571,0.212362,0.2524,0.598558,0.133345,0.486667,0.614557,0.088233,0.815756,0.10420,0.852000,0.849175,0.4272,0.4668,0.272800,0.319933,0.326250,0.320402,0.898421,0.606896,0.966000,0.843241,0.847308,0.807108,0.819775,1.000000,1.0,0.286070,0.200193,0.960060,-0.9762,0.221725,0.0,0.868441,0.886859,0.322187,0.1600,72.0,0.829908,0.624072,0.832753,111.0,0.276924,0.23728,14.769169,0.618157,0.205543,0.802174,0.845714,0.5450,13.677975,0.4884,0.35851,0.134215,0
515429,2015-12-05 00:30:00.000003,34,1211748,0.252293,0.055179,0.419899,0.186505,0.530000,0.032,0.026,0.438824,-0.062723,0.069277,-0.124722,-0.018389,0.049944,0.086611,0.129267,-0.058598,-0.02482,-0.01172,0.832454,0.362286,0.217166,0.2536,0.597116,0.133483,0.483030,0.616013,0.088233,0.822994,0.08825,0.851429,0.849776,0.4644,0.4668,0.266686,0.325871,0.333571,0.326451,0.898947,0.607422,0.982667,0.842443,0.846310,0.807510,0.819910,1.000401,1.0,0.286790,0.188440,0.960060,-0.9613,0.195735,0.0,0.868321,0.886859,0.328638,0.1600,72.0,0.828776,0.623482,0.832454,111.0,0.246152,0.24264,14.790450,0.618336,0.209571,0.803043,0.849714,0.5450,13.780506,0.4856,0.36771,0.134215,0
515430,2015-12-05 00:39:59.999997,34,1211749,0.299697,0.115222,0.436924,0.185084,0.530000,0.032,0.016,0.438824,-0.063444,0.064889,-0.132111,-0.007444,0.054889,0.083222,0.090337,-0.042950,-0.01256,-0.00842,0.833556,0.369143,0.224172,0.2584,0.597116,0.133689,0.481818,0.616646,0.088233,0.819892,0.10175,0.856571,0.850201,0.4436,0.4728,0.257429,0.325714,0.334375,0.327433,0.899474,0.608590,0.988667,0.844534,0.846534,0.807134,0.820868,1.000201,1.0,0.286712,0.192200,0.960429,-0.9608,0.228222,0.0,0.868627,0.886859,0.329152,0.1590,72.0,0.832068,0.622301,0.833556,111.0,0.289744,0.23900,14.743637,0.616186,0.217943,0.806957,0.852000,0.5445,13.882785,0.4780,0.37120,0.134215,0
